In [3]:
import numpy as np

In [4]:
# Loading the pose training and validating data
trainX = np.load("training/finaltraining_data.npy", allow_pickle=True)
trainY = np.load("training/finaltraininglabels_data.npy", allow_pickle=True)
testX = np.load("training/validation_data.npy", allow_pickle=True)
testY = np.load("training/validation_data_labels.npy", allow_pickle=True)

In [937]:
trainX.shape

(502, 60, 25, 2)

In [938]:
trainY.shape

(502, 60, 4)

In [939]:
testY.shape

(242, 60, 4)

In [940]:
testX.shape

(242, 60, 25, 2)

In [5]:
# CNN + LSTM model
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers.pooling import GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, LSTM
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from keras.layers import Conv2D
from keras.layers import MaxPool1D, MaxPool2D
import tensorflow as tf

cnn = Sequential()
cnn.add(TimeDistributed(keras.layers.Conv1D(filters=64, kernel_size=3,padding='same', activation='relu'), input_shape=trainX.shape[1:]))
cnn.add(TimeDistributed(BatchNormalization()))
cnn.add(TimeDistributed(MaxPooling1D(2,2)))
cnn.add(TimeDistributed(keras.layers.Conv1D(filters=64, kernel_size=3,padding='same', activation='relu'), input_shape=trainX.shape[1:]))
cnn.add(TimeDistributed(BatchNormalization()))
cnn.add(TimeDistributed(MaxPooling1D(2,2)))
cnn.add(TimeDistributed(Dropout(0.5)))
cnn.add(BatchNormalization())
cnn.add(TimeDistributed(Flatten()))
cnn.add(LSTM(40,unit_forget_bias=0.4, return_sequences=True))
cnn.add(TimeDistributed(Dense(4, activation='softmax')))
cnn.summary()
print(cnn.output_shape)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 60, 25, 64)        448       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 60, 25, 64)        256       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 60, 12, 64)        0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 60, 12, 64)        12352     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 60, 12, 64)        256       
_________________________________________________________________
time_distributed_5 (TimeDist (None, 60, 6, 64)         0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 60, 6, 64)        

In [8]:
# Compiling the model
adam = Adam(lr=0.0001)

cnn.compile(loss='categorical_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])

In [9]:
# history = cnn.fit(trainX, trainY, validation_data = (testX,testY), epochs=200)

In [10]:
# Load the weights
cnn.load_weights("weights/final_model.h5")

In [11]:
# validating the model on training and validation data
_, train_acc = cnn.evaluate(trainX, trainY)
_, test_acc = cnn.evaluate(testX, testY)
print(train_acc)
print(test_acc)

8/8 [==============================] - 0s 20ms/step - loss: 0.0670 - accuracy: 0.9799
0.9766932129859924
0.9798898100852966


In [12]:
# Loading a testing throwing pose data
x_test = np.load("data/shi_throw.npy", allow_pickle=True)
x_test = x_test[:240,...]
x_test = np.asarray(x_test).astype(np.float32)
x_test = x_test.reshape((4,60,25,2))

In [13]:
motion = ["knocking", "lifting", "throwing", "walking"]

In [14]:
Yht_pred = cnn.predict(np.asarray(x_test))
a = np.argmax(Yht_pred[0],axis = 1)
(np.argmax(Yht_pred[1],axis = 1))

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [ ]:
# The results above shows the predicted frames with labels and can also be used like in the cell below

In [15]:
import statistics
motion[statistics.mode(np.argmax(Yht_pred[1],axis = 1))]

'throwing'